In [284]:
import pubchempy as pcp
import numpy as np
import pandas as pd
import xlrd
import ast
import csv
import opentrons
import datetime
import importlib
import setup_functions
importlib.reload(setup_functions)

<module 'setup_functions' from 'C:\\Users\\lacho\\Documents\\Pozzo-RG-OT2\\OT2Protocols\\code-collab\\database\\setup_functions.py'>

In [285]:
experiment_plan_filepath = 'experiment_plan.csv'
experiment_plan_dict = setup_functions.get_experiment_plan(experiment_plan_filepath)
experiment_plan_dict
chemical_db_df = pd.read_excel('chemical_inventory.xlsx')
chemical_db_df

,Chemical Name,Chemical Abbreviation,Molecular Weight (g/mol),Density (g/mL),InChI=,InChI Key (ID),Chemical Category,Canonical SMILES String,Molecular Formula,PubChem ID,Catalog description,Synonyms,Catalog number,Purchase URL,Primary Information Source
0,Water,Water,18.015,0.995,1S/H2O/h1H2,XLYOFNOQVPJJNP-UHFFFAOYSA-N,solvent,O,H2O,962,NaN,NaN,NaN,NaN,NaN
1,Sodium Chloride,NaCl,58.440,2.165,1S/ClH.Na/h1H;/q;+1/p-1,FAPWRFPIFSIZLT-UHFFFAOYSA-M,"inorganic, halide",[Na+].[Cl-],NaCl,5234,NaN,NaN,NaN,NaN,NaN
2,Butenedioic acid,Butenedioicacid,116.070,1.500,"1S/C4H4O4/c5-3(6)1-2-4(7)8/h1-2H,(H,5,6)(H,7,8)",VZCYOOQTPOCHFL-UHFFFAOYSA-N,"hbd, acid",C(=CC(=O)O)C(=O)O,C4H4O4,723,NaN,NaN,NaN,NaN,NaN
3,Choline chloride,chcl,139.620,NaN,"1S/C5H14NO.ClH/c1-6(2,3)4-5-7;/h7H,4-5H2,1-3H3...",SGMZJAMFUVOLNK-UHFFFAOYSA-M,"hba, organic, halide",C[N+](C)(C)CCO.[Cl-],C5H14ClNO,6209,NaN,NaN,NaN,NaN,NaN


In [282]:
volf_array = setup_functions.generate_candidate_samples(experiment_plan_dict, chemical_db_df)

In [283]:
volf_array

array([[2.69930716e-05, 7.73800000e-05, 1.39620000e-04, 9.99756007e-01],
       [2.69930716e-05, 7.73800000e-05, 7.48069263e-03, 9.92414934e-01],
       [2.69930716e-05, 7.73800000e-05, 1.48217653e-02, 9.85073862e-01],
       ...,
       [2.69930716e-02, 7.73800000e-02, 1.24937855e-01, 7.70689074e-01],
       [2.69930716e-02, 7.73800000e-02, 1.32278927e-01, 7.63348001e-01],
       [2.69930716e-02, 7.73800000e-02, 1.39620000e-01, 7.56006928e-01]])

In [241]:
conc_array = conc_df.values
selection = chemical_db_df['Chemical Abbreviation'] == 'NaCl'
setup_functions.concconvert(conc_array[:,0], 'molarity', chemical_db_df[selection],'volf').shape

(8000,)

In [28]:
component_df = pd.read_excel('chemical_inventory.xlsx')

component_list = ['NaCl']
selection = component_df['Chemical Abbreviation'].isin(component_list)
component_df = component_df[selection]


component_df

,Chemical Name,Chemical Abbreviation,Molecular Weight (g/mol),Density (g/mL),InChI=,InChI Key (ID),Chemical Category,Canonical SMILES String,Molecular Formula,PubChem ID,Catalog description,Synonyms,Catalog number,Purchase URL,Primary Information Source
1,Sodium Chloride,NaCl,58.44,2.165,1S/ClH.Na/h1H;/q;+1/p-1,FAPWRFPIFSIZLT-UHFFFAOYSA-M,"inorganic, halide",[Na+].[Cl-],NaCl,5234,NaN,NaN,NaN,NaN,NaN


In [216]:
stock_spec_filepath = 'stock_specification.csv'
stock_dict = setup_functions.get_stock_dfs(stock_spec_filepath)
stock_dict['hba-stock']

,State,Chemical Abbreviation,Order,Maximum Concentration,Stock 1 Nominal Concentration,Stock 1 Experimental Concentration,Concentration Units
1,solid,NaCl,primary,5,3,,M
2,solid,Butenedioicacid,primary,1,1,,M
3,liquid,Water,secondary,,1,,volume fraction


In [220]:
conc_df = setup_functions.generate_candidate_samples(experiment_plan_dict)
conc_df

,NaCl,Butenedioicacid,chcl,Water
0,0.001,0.001,0.001000,1.0
1,0.001,0.001,0.053579,1.0
2,0.001,0.001,0.106158,1.0
3,0.001,0.001,0.158737,1.0
4,0.001,0.001,0.211316,1.0
...,...,...,...,...
7995,1.000,1.000,0.789684,1.0
7996,1.000,1.000,0.842263,1.0
7997,1.000,1.000,0.894842,1.0
7998,1.000,1.000,0.947421,1.0


In [ ]:
def cull_candidate_samples(experiment_plan_dict, stock_dict):
    """
    Attempt to take care of components appearing in more than one set.
    Pretty, but difficult to execute.
    """
    start_df = setup_functions.generate_candidate_samples(experiment_plan_dict)
    all_component_names_set = set()
    # Generate set of all unique names
    for k,v in stock_dict.items():
        all_component_names_set.update(v['Chemical Abbreviation'].values)
    print(all_component_names_set,'\n')
    # Specify which componennt is unique to each stock. 
    for k,v in stock_dict.items():
        name_set = set(v['Chemical Abbreviation'].values)
        not_in_set = all_component_names_set-name_set
#         print(name_set)
#         print(not_in_set)
#         print("\n")

def cull_candidate_samples(experiment_plan_dict, stock_dict):
    start_df = setup_functions.generate_candidate_samples(experiment_plan_dict) 
    for stock_name in experiment_plan_dict['Stock order']:
        stock_df = stock_dict[stock_name]
        print(stock_name)
        print(stock_df['Chemical Abbreviation'].values)
        print('\n')
cull_candidate_samples(experiment_plan_dict, stock_dict)

In [74]:
class TestClass:
    def __init__(self,y):
        self.x = 2
        self.y = y

vTestClass = np.vectorize(TestClass)
test_array = np.empty((3,3), dtype=object)
test_array[:,:] = vTestClass(2)


2

In [ ]:
array_of_objects = np.empty(shape=(10,10), dtype=object)
array_of_objects[:,:] = new_object
array_of_objects

In [ ]:
conc_df.values